In [1]:
!pip install pytorch-nlp
!pip install datasets
!pip install transformers
!pip install sentencepiece
!pip install tqdm
!pip install peft
!pip install trl
!pip install evaluate

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [2]:
from transformers import T5ForConditionalGeneration, T5Tokenizer, AdamW
from datasets import load_dataset
from torch.utils.data import DataLoader, Dataset
import torch

from transformers import pipeline, AutoTokenizer, AutoModelForSequenceClassification, AutoModelForSeq2SeqLM, GenerationConfig
from datasets import load_dataset
from peft import PeftModel, PeftConfig, LoraConfig, TaskType

# trl: Transformer Reinforcement Learning library
from trl import PPOTrainer, PPOConfig, AutoModelForSeq2SeqLMWithValueHead
from trl import create_reference_model
from trl.core import LengthSampler

import evaluate

import numpy as np
import pandas as pd

from transformers import AutoTokenizer
from trl import PPOTrainer, PPOConfig, AutoModelForCausalLMWithValueHead, create_reference_model
from trl.core import respond_to_batch

from tqdm import tqdm

In [3]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"
!pip install rouge_score
!pip install bert_score
!pip install rouge

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [4]:
# Load SQuAD dataset
squad_data = load_dataset("squad_v2")['train']

Found cached dataset squad_v2 (/home/rpapani/.cache/huggingface/datasets/squad_v2/squad_v2/2.0.0/09187c73c1b837c95d9a249cd97c2c3f1cebada06efe667b4427714b27639b1d)


  0%|          | 0/2 [00:00<?, ?it/s]

In [5]:
#extracting context
c_temp = squad_data[0]['context']
#extracting answer
ans_temp = squad_data[0]['answers']['text'][0]
print(c_temp)

Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ bee-YON-say) (born September 4, 1981) is an American singer, songwriter, record producer and actress. Born and raised in Houston, Texas, she performed in various singing and dancing competitions as a child, and rose to fame in the late 1990s as lead singer of R&B girl-group Destiny's Child. Managed by her father, Mathew Knowles, the group became one of the world's best-selling girl groups of all time. Their hiatus saw the release of Beyoncé's debut album, Dangerously in Love (2003), which established her as a solo artist worldwide, earned five Grammy Awards and featured the Billboard Hot 100 number-one singles "Crazy in Love" and "Baby Boy".


In [6]:
# Prepare the dataset for fine-tuning
class SquadDataset(Dataset):
    def __init__(self, data):
        self.data = [example for example in data if example['answers']['text']]

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        example = self.data[idx]
        context = example['context']
        answer = example['answers']['text'][0]  # Assuming only one answer for simplicity
        question = example['question']
        return context, "\n Question for the above generated context is : " + question

In [7]:
tokenizer = T5Tokenizer.from_pretrained('t5-small')
model = T5ForConditionalGeneration.from_pretrained('t5-small')

# Prepare the dataset
context = SquadDataset(squad_data)
print(context.__getitem__(0))

('Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ bee-YON-say) (born September 4, 1981) is an American singer, songwriter, record producer and actress. Born and raised in Houston, Texas, she performed in various singing and dancing competitions as a child, and rose to fame in the late 1990s as lead singer of R&B girl-group Destiny\'s Child. Managed by her father, Mathew Knowles, the group became one of the world\'s best-selling girl groups of all time. Their hiatus saw the release of Beyoncé\'s debut album, Dangerously in Love (2003), which established her as a solo artist worldwide, earned five Grammy Awards and featured the Billboard Hot 100 number-one singles "Crazy in Love" and "Baby Boy".', '\n Question for the above generated context is : When did Beyonce start becoming popular?')


In [8]:
def collate_fn(batch):
    contexts, questions = zip(*batch)
    # print("Print Context",contexts)
    # print("Print Context", questions)
    inputs = tokenizer(contexts, return_tensors='pt', padding=True, truncation=True)
    labels = tokenizer(questions, return_tensors='pt', padding=True, truncation=True)
    return {**inputs, 'labels': labels['input_ids']}

In [9]:
dataloader = DataLoader(context, batch_size=50, shuffle=True, collate_fn=collate_fn)

In [10]:
optimizer = torch.optim.AdamW(model.parameters(), lr=5e-4)

In [11]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
device

device(type='cuda')

In [12]:
def run_model(input_string, **generator_args):
    input_ids = tokenizer.encode(input_string, return_tensors="pt")
    res = model.generate(input_ids, **generator_args)
    output = tokenizer.batch_decode(res, skip_special_tokens=True)
    #print(output)
    return output

In [13]:
tokenizer = T5Tokenizer.from_pretrained("allenai/t5-small-squad2-question-generation")
model = AutoModelForSeq2SeqLMWithValueHead.from_pretrained("allenai/t5-small-squad2-question-generation")
model_ref = AutoModelForSeq2SeqLMWithValueHead.from_pretrained("allenai/t5-small-squad2-question-generation")
tokenizer.pad_token = tokenizer.eos_token

Detected kernel version 4.18.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.
Detected kernel version 4.18.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


In [14]:
# 2. initialize trainer
ppo_config = {"batch_size": 50}
config = PPOConfig(**ppo_config)
ppo_trainer = PPOTrainer(config, model, model_ref, tokenizer, data_collator=collate_fn)

Detected kernel version 4.18.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.
/home/rpapani/.local/lib/python3.10/site-packages/trl/trainer/ppo_trainer.py:226: UserWarning: No dataset is provided. Make sure to set config.batch_size to the correct value before training.
  warnings.warn(


In [15]:
#query text in batches
query_txt = "He thanked all fellow bloggers and organizations that showed support."
references = [["Who did he thank?"]]
query_tensor = tokenizer.encode(query_txt, return_tensors="pt").to(model.pretrained_model.device)

In [16]:
generation_kwargs = {
    "top_k": 0.0, # no top-k sampling
    "top_p": 1.0, # no nucleus sampling
    "do_sample": True, # yes, we want to sample
    "max_new_tokens": 32, # specify how many tokens you want to generate at most
}
# response_tensor = ppo_trainer.generate([item for item in query_tensor], return_prompt=False, **generation_kwargs)
# response_txt = tokenizer.decode(response_tensor[0])
# output = tokenizer.batch_decode(response_tensor, skip_special_tokens=True)
# print(output)
# print(response_txt)

In [17]:
# bleu = evaluate.load("bleu")
# results = bleu.compute(predictions=output, references=references)
# print(results)
# print(results.get("bleu"))
# bleu_score = results.get("bleu")

In [18]:
# reward = [torch.tensor(bleu_score)]

In [19]:
# num_epochs=3
# bt={}
# response_tensors = []
# reft=[]
# qns=[]
# for epoch in tqdm(range(num_epochs), desc="Epochs"):
#     for batch_idx, batch in tqdm(enumerate(dataloader), total=len(dataloader), desc="Batches", leave=False):
#         if(batch_idx<=1):
#           reward=[]
#           query_tensors = batch["input_ids"]
#           reft.append(batch["labels"])
#           response_tensors = ppo_trainer.generate([item for item in query_tensors], return_prompt=False, **generation_kwargs)
#           batch["response"] = tokenizer.batch_decode(response_tensors, skip_special_tokens=True)
#           batch["ref"] = [tokenizer.batch_decode(k, skip_special_tokens=True)for k in reft]
#           batch["context"] = [tokenizer.batch_decode(p, skip_special_tokens=True) for p in query_tensor]
#           bleu = evaluate.load("bleu")
#           predictions_and_references = {'predictions': batch["response"], 'references': batch["ref"][0]}
#           results = bleu.compute(predictions=predictions_and_references["predictions"], references=predictions_and_references["references"])
#           bleu_score = results.get("bleu")
#           reward.append(torch.tensor(bleu_score))
#           train_stats = ppo_trainer.step(query_tensors, response_tensors, reward)
#           ppo_trainer.log_stats(train_stats, batch, reward)
#         else:
#           break

In [20]:
# Use a pipeline as a high-level helper
from transformers import pipeline

reward_model = pipeline("translation", model="t5-small")

/home/rpapani/.local/lib/python3.10/site-packages/transformers/pipelines/__init__.py:955: UserWarning: "translation" task was used, instead of "translation_XX_to_YY", defaulting to "translation_en_to_de"
  warnings.warn(


In [21]:
from rouge import Rouge
from bert_score import score as bert_score
from nltk.translate.bleu_score import sentence_bleu
from nltk.translate.meteor_score import meteor_score

generation_kwargs = {
    "min_new_tokens": 1, # don't ignore the EOS token (see above)
    "top_k": 0.0, # no top-k sampling
    "top_p": 1.0, # no nucleus sampling
    "do_sample": True, # yes, we want to sample
    "pad_token_id": tokenizer.eos_token_id, # most decoder models don't have a padding token - use EOS token instead
    "max_new_tokens": 32, # specify how many tokens you want to generate at most
}
num_epochs = 4
reward_coefficients = {'rouge': 0.4, 'bert_score': 0.3, 'bleu': 0.2, 'meteor': 0.1}
reft = []
meteor = evaluate.load("meteor")
rouge = evaluate.load("rouge")
bertscore = evaluate.load("bertscore")
for epoch in tqdm(range(num_epochs), desc="Epochs"):
    for batch_idx, batch in tqdm(enumerate(dataloader), total=len(dataloader), desc="Batches", leave=False):
        if batch_idx == 2:
            # reward = []
            query_tensors = batch["input_ids"]
            reft.append(batch["labels"])
            response_tensors = ppo_trainer.generate([item for item in query_tensors], return_prompt=False, **generation_kwargs)
            batch["response"] = tokenizer.batch_decode(response_tensors, skip_special_tokens=True)
            batch["ref"] = [tokenizer.batch_decode(k, skip_special_tokens=True) for k in reft]
            predictions_and_references = {'predictions': batch["response"], 'references': batch["ref"][0]}
            rouge_scores = []

            # indices_with_all_padding = [idx for idx, tensor in enumerate(response_tensors) if all(element == padding_token_id for element in tensor.flatten())]

            # print("Indices with all padding tokens:", indices_with_all_padding)
            rewardLen =len(predictions_and_references["predictions"])
            # for i in range(rewardLen):
            #     prediction = predictions_and_references["predictions"][i]
            #     reference = predictions_and_references["references"][i]
            #     rouge_score = rouge.compute(predictions=[prediction], references=[[reference]]).get("rougeL")
            #     rouge_scores.append(torch.tensor(rouge_score))
                # print("Iteration of Rouge : ", i)
                
            # meteor_scores = []
            # for i in range(rewardLen):
            #     prediction = predictions_and_references["predictions"][i]
            #     reference = predictions_and_references["references"][i]
            #     meteor_score = meteor.compute(predictions=[prediction], references=[[reference]]).get("meteor")
            #     meteor_scores.append(torch.tensor(meteor_score))
            #     # print("Iteration of Meteor : ", i)

            bertscores=[]
            for i in range(rewardLen):
                prediction = predictions_and_references["predictions"][i]
                reference = predictions_and_references["references"][i]
                bertscore_score = np.mean(bertscore.compute(predictions=[prediction], references=[[reference]], lang='en', verbose=True).get("precision"))
                bertscores.append(torch.tensor(bertscore_score))

            #individual score reward calc
            # average_scores = [((rouge) ).clone().detach() for rouge, meteor, bertscore in zip(rouge_scores, meteor_scores, bertscores)]
            
            # average_scores = [((rouge + meteor+bertscore) / 3).clone().detach() for rouge, meteor, bertscore in zip(rouge_scores, meteor_scores, bertscores)]
            
            # print(average_scores)


            # # ROUGE
            # rouge = evaluate.load("rouge")
            # rouge_results = rouge.compute(predictions=predictions_and_references["predictions"], references=predictions_and_references["references"])
            # rouge_score = rouge_results.get("rougeL")

            # # METEOR Score
            # meteor = evaluate.load("meteor")
            # meteor_results = meteor.compute(predictions=predictions_and_references["predictions"], references=predictions_and_references["references"])
            # meteor_score = meteor_results.get("meteor")

            # # BERT Score
            # bertscore = evaluate.load("bertscore")
            # bertscore_results = bertscore.compute(predictions=predictions_and_references["predictions"], references=predictions_and_references["references"], lang='en', verbose=True)
            # bertscore_score = np.mean(bertscore_results.get("precision"))
            # print(meteor_results)
            # # Combine Scores
            # print("Reward cofficient Meteor: ", reward_coefficients['meteor'] , " And " , meteor_score)
            # total_reward = (
            #     reward_coefficients['rouge'] * rouge_score +
            #     reward_coefficients['bert_score'] * bertscore_score +
            #     reward_coefficients['bleu'] * bleu_score +
            #     reward_coefficients['meteor'] * meteor_score
            # )

            # list_of_tensors = [torch.tensor(sublist) for sublist in query_tensors]
            list_of_tensors_lists = [sublist.clone().detach() for sublist in query_tensors]
            # print("This is :", list_of_tensors_lists)
            # print(response_tensors)
            # print(len(list_of_tensors))
            # print(len(response_tensors))
            # print(len(meteor_scores))
            # print(len(response_tensors), response_tensors[0].shape)
            # print(len(list_of_tensors_lists), list_of_tensors_lists[0].shape)
            # [print("Query Tensor",tensor_) for tensor_ in list_of_tensors_lists if tensor_.numel() ==0]
            # [print("Response Tensor",tensor_) for tensor_ in response_tensors if tensor_.numel() ==0]
            # [print(tensor_.shape) for tensor_ in response_tensors]
            train_stats = ppo_trainer.step(list_of_tensors_lists, response_tensors, bertscores)
            # ppo_trainer.log_stats(train_stats, batch, reward)
        elif batch_idx <2:
            continue
        else:
            break    

[nltk_data] Downloading package wordnet to /home/rpapani/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /home/rpapani/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /home/rpapani/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
Batches:   0%|          | 0/1737 [00:00<?, ?it/s]Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


calculating scores...
computing bert embedding.


  0%|          | 0/1 [00:00<?, ?it/s]

computing greedy matching.


  0%|          | 0/1 [00:00<?, ?it/s]

...

  0%|          | 0/1 [00:00<?, ?it/s]

computing greedy matching.


  0%|          | 0/1 [00:00<?, ?it/s]

done in 4337644.65 seconds, 0.00 sentences/sec


/home/rpapani/.local/lib/python3.10/site-packages/trl/trainer/ppo_trainer.py:1091: UserWarning: KL divergence is starting to become negative: -4.50 - this might be a precursor for failed training. sometimes this happens because the generation kwargs are not correctly set. Please make sure that the generation kwargs are set correctly, or review your training hyperparameters.
  warnings.warn(

Epochs: 100%|██████████| 4/4 [01:47<00:00, 26.94s/it]      


In [34]:
model.save_pretrained("Reward_function_bert")

In [38]:
from transformers import T5ForConditionalGeneration
model_name = "Reward_function_bert"
model = T5ForConditionalGeneration.from_pretrained(model_name)

In [39]:
tokenizer = T5Tokenizer.from_pretrained("allenai/t5-small-squad2-question-generation")
tokenizer.pad_token = tokenizer.eos_token

In [40]:
input_text = c_temp
input_ids = tokenizer.encode(input_text, return_tensors="pt")
output_ids = model.generate(input_ids)
output_text = tokenizer.decode(output_ids[0], skip_special_tokens=True)
print(f"context:{c_temp} \n question generated:{output_text}")

context:Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ bee-YON-say) (born September 4, 1981) is an American singer, songwriter, record producer and actress. Born and raised in Houston, Texas, she performed in various singing and dancing competitions as a child, and rose to fame in the late 1990s as lead singer of R&B girl-group Destiny's Child. Managed by her father, Mathew Knowles, the group became one of the world's best-selling girl groups of all time. Their hiatus saw the release of Beyoncé's debut album, Dangerously in Love (2003), which established her as a solo artist worldwide, earned five Grammy Awards and featured the Billboard Hot 100 number-one singles "Crazy in Love" and "Baby Boy". 
 question generated:What song did Beyonce's debut album, Dangerously in Love, feature?


In [41]:
import evaluate
# Get meteor
meteor = evaluate.load("meteor")
results = meteor.compute(predictions=[output_text], references=[squad_data[0]['question']])
print(results)
print(results.get("meteor"))
meteor_score = results.get("meteor")

{'meteor': 0.3318903318903319}
0.3318903318903319


[nltk_data] Downloading package wordnet to /home/rpapani/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /home/rpapani/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /home/rpapani/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [42]:
test_data = load_dataset("squad_v2")['validation']

  0%|          | 0/2 [00:00<?, ?it/s]

In [43]:
test_data[0]['context']

'The Normans (Norman: Nourmands; French: Normands; Latin: Normanni) were the people who in the 10th and 11th centuries gave their name to Normandy, a region in France. They were descended from Norse ("Norman" comes from "Norseman") raiders and pirates from Denmark, Iceland and Norway who, under their leader Rollo, agreed to swear fealty to King Charles III of West Francia. Through generations of assimilation and mixing with the native Frankish and Roman-Gaulish populations, their descendants would gradually merge with the Carolingian-based cultures of West Francia. The distinct cultural and ethnic identity of the Normans emerged initially in the first half of the 10th century, and it continued to evolve over the succeeding centuries.'

In [44]:
#test context data
tcd = []
trq=[]
for i in range(500):
    tcd.append(test_data[i]['context'])
    trq.append(test_data[i]['question'])

In [45]:
# Get meteor

meteor = evaluate.load("meteor")
rouge = evaluate.load("rouge")
bertscore = evaluate.load("bertscore")
meteor_res=[]
rouge_res=[]
bertscore_res=[]
for i in range(500):
    meteor_res.append(meteor.compute(predictions=[tcd[i]], references=[trq[i]]).get('meteor'))
    rouge_res.append(rouge.compute(predictions=[tcd[i]], references=[trq[i]]).get('rougeL'))
    bertscore_res.append( np.mean(bertscore.compute(predictions=[tcd[i]], references=[trq[i]], lang='en', verbose=True).get("precision")))
#print(bertscore_res)
# print(results.get("meteor"))
# meteor_score = results.get("meteor")

[nltk_data] Downloading package wordnet to /home/rpapani/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /home/rpapani/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /home/rpapani/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


calculating scores...
computing bert embedding.


  0%|          | 0/1 [00:00<?, ?it/s]

computing greedy matching.


  0%|          | 0/1 [00:00<?, ?it/s]

...

done in 4337909.51 seconds, 0.00 sentences/sec
calculating scores...
computing bert embedding.


  0%|          | 0/1 [00:00<?, ?it/s]

computing greedy matching.


  0%|          | 0/1 [00:00<?, ?it/s]

done in 4337909.73 seconds, 0.00 sentences/sec


In [35]:
np.mean(meteor_res)

0.21925562843353355

In [36]:
np.mean(rouge_res)

0.10892931195743866

In [37]:
np.mean(bertscore_res)

0.8104894465208053